In [1]:
import numpy as np
import numba as nb
import scipy
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from scipy.fft import fft, ifft, fft2, ifft2
from scipy.fft import dst,  idst, fft, ifft, fftshift, ifftshift, fftfreq
from scipy.linalg import polar
from math import sqrt, tanh, ceil, floor, log10, exp, cos, sin, copysign
import os
import re





def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)




pi = np.pi
π = np.pi

%run plotting.ipynb
%run eulerianGrid.ipynb
%run lagrangianParticles.ipynb
%run levelSet.ipynb


@nb.jit(nopython=True, fastmath=True)  
def N(x):
    d = abs(x)
    if d < 1.:
        return (.5*d - 1.)*d*d + 2./3.
    elif d < 2.:
        return ((-1./6.*d + 1)*d-2.)*d + 4./3.
    else:
        return 0.
    
@nb.jit(nopython=True, parallel=True, fastmath=True, cache=True)
def substep1(X, U, M,
             xx, yy, zz, mm, uu, vv, ww, hx, hy, hz, 
             ib, jb, kb, it, jt, kt):
    for n in range(M.shape[0]):
        for i in nb.prange(ib[n], it[n]):
            w0 = N( ( X[n,0]-xx[i,0,0] ) / hx )
            for j in range(jb[n], jt[n]):
                w1 = N( ( X[n,1]-yy[0,j,0] ) / hy )
                for k in range(kb[n], kt[n]):
                    w = w0 * w1 * N( ( X[n,2]-zz[0,0,k] ) / hz ) 
                    w *= M[n, 0]
                    mm[i,j,k] += w  
                    
def step1(eg, lp):
    eg.mm[:] = 0.
    eg.uu[:] = 0.
    eg.vv[:] = 0.
    eg.ww[:] = 0.

    ic = np.floor( (lp.X[:,0]-eg.xx[0,0,0])/eg.hx ).astype(int)
    jc = np.floor( (lp.X[:,1]-eg.yy[0,0,0])/eg.hy ).astype(int)
    kc = np.floor( (lp.X[:,2]-eg.zz[0,0,0])/eg.hz ).astype(int)

    ib = np.maximum(0, (ic[:]-2) % Nx)
    jb = np.maximum(0, (jc[:]-2) % Ny)
    kb = np.maximum(0, (kc[:]-2) % Nz)

    it = np.minimum(Nx-1, (ic[:]+3) % Nx)
    jt = np.minimum(Ny-1, (jc[:]+3) % Ny)
    kt = np.minimum(Nz-1, (kc[:]+3) % Nz)

    substep1(lp.X, lp.U, lp.M,
             eg.xx, eg.yy, eg.zz, eg.mm, eg.uu, eg.vv, eg.ww, eg.hx, eg.hy, eg.hz,
             ib, jb, kb, it, jt, kt)
    
def writeData( pathnameOriginal, mm, s ):
    Nx, Ny, Nz = mm.shape
    with open( pathnameOriginal, 'r' ) as file:
        fileRead = file.readlines() 
    with open( 'steps/step'+str(s)+'.pbrt', 'w') as f:
        for i, line in enumerate(fileRead):
            if i < 3:
                f.write(line)
            else:
                break
        f.write("\n".join(" ".join(map(str, mm[i])) for i in np.ndindex(mm.shape[1:])  ))  
        f.write(']')
        f.close()

    


In [2]:
xpath = r'/Users/daniellam/Desktop/materialPointOptimizedCopy/case1/X0.npy'
X = np.load(xpath)
upath = r'/Users/daniellam/Desktop/materialPointOptimizedCopy/case1/U0.npy'
mpath = r'/Users/daniellam/Desktop/materialPointOptimizedCopy/case1/M0.npy'

g = 9.81
Nx = 50
Ny = 50
Nz = 50
numberParticles = X.shape[0]


eg = eulerianGrid(Nx, Ny, Nz)
lp = lagrangianParticles(numberParticles)

lp.loadData(xpath, upath, mpath)

step1(eg, lp)



In [3]:
pathnameOriginal = r'/Users/daniellam/Downloads/pbrscenes/pbrt-v3-scenes/snow/geometry/density_render.70.pbrt'



writeData(pathnameOriginal, eg.mm)

print(eg.mm.shape)


TypeError: writeData() missing 1 required positional argument: 's'

In [ ]:
a = np.array([1.,2,3])
b = np.array([4.,5.])
filename='waldo.txt'
with open(filename,"w") as f:
    f.write("\n".join(" ".join(map(str, x)) for x in (a,b)))
    
print(a)
for x in (a,b):
    print(x)

In [4]:
pathname = r"/Users/daniellam/Desktop/materialPointOptimizedCopy/case1"
pathnameOriginal = r'/Users/daniellam/Downloads/pbrscenes/pbrt-v3-scenes/snow/geometry/density_render.70.pbrt'
pathnameOriginal2 = "/Users/daniellam/Downloads/pbrscenes/pbrt-v3-scenes/snow/smoke.pbrt"


filenames = []
    
for subdir, dirs, files in os.walk(pathname):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath[-3:] == 'npy':
                filenames.append(filepath)


sort_nicely(filenames)
N = len(filenames)//3

xpaths = []
upaths = []
mpaths = []


for i, path in enumerate(filenames):
    if i//N == 0:
        mpaths.append(path)
    if i//N == 1:
        upaths.append(path)
    if i//N == 2:
        xpaths.append(path)


g = 9.81
Nx = 50
Ny = 50
Nz = 50
eg = eulerianGrid(Nx, Ny, Nz)


i = 0
for xpath, upath, mpath in zip(xpaths, upaths, mpaths):
    try:
        lp
    except NameError:
        ## initialize lp
        X = np.load(xpath)
        numberParticles = X.shape[0]
        lp = lagrangianParticles(numberParticles)
        
        lp.loadData(xpath, upath, mpath)
        step1(eg, lp)
        writeData(pathnameOriginal, eg.mm, i)
        replacePbrt( pathnameOriginal2, i )
    else:        
        lp.loadData(xpath, upath, mpath)
        step1(eg, lp)
        writeData(pathnameOriginal, eg.mm, i)
        replacePbrt( pathnameOriginal2, i )
        
    print('pbrt ../../pbrt-v3-scenes/cloud/snow' + str(i) + '.pbrt' )
    i += 1







NameError: name 'replacePbrt' is not defined

In [5]:
def writeData( pathnameOriginal, mm, s ):
    Nx, Ny, Nz = mm.shape
    with open( pathnameOriginal, 'r' ) as file:
        fileRead = file.readlines() 
    with open( 'steps/step'+str(s)+'.pbrt', 'w') as f:
        for i, line in enumerate(fileRead):
            if i < 3:
                f.write(line)
            else:
                break
        rearranged_arr = np.moveaxis(mm, [0, 1, 2], [2, 1, 0])
        f.write("\n".join(" ".join(map(str, rearranged_arr[i])) for i in np.ndindex(mm.shape[1:])  ))  
        f.write(']')
        f.close()
        
def replacePbrt( pathnameOriginal, s ):
    wordsToReplace = ['density_render.70.pbrt', 'smoke.exr'         ]
    replacement    = ['step'+str(s)+'.pbrt',    'step'+str(s)+'.exr']
    
    with open( pathnameOriginal, 'r' ) as file:
        fileRead = file.readlines() 
    with open( 'pbrt/snow'+str(s)+'.pbrt', 'w') as f:
        for i, line in enumerate(fileRead):
            for j, word in enumerate(wordsToReplace):
                if word in line:
                    line = line.replace(word, replacement[j]) 
            f.write(line)
                
replacePbrt( pathnameOriginal, 0 )

In [6]:
def writeData( pathnameOriginal, mm, s ):
    Nx, Ny, Nz = mm.shape
    with open( pathnameOriginal, 'r' ) as file:
        fileRead = file.readlines() 
    with open( 'steps/step'+str(s)+'.pbrt', 'w') as f:
        for i, line in enumerate(fileRead):
            if i < 3:
                f.write(line)
            else:
                break
        rearranged_arr = np.moveaxis(mm, [0, 1, 2], [2, 1, 0])
        f.write("\n".join(" ".join(map(str, rearranged_arr[i])) for i in np.ndindex(mm.shape[1:])  ))  
        f.write(']')
        f.close()
        
def replacePbrt( pathnameOriginal, s ):
    wordsToReplace = ['step30.pbrt', 'step41.exr'         ]
    replacement    = ['step'+str(s)+'.pbrt',    'step'+str(s)+'.exr']
    
    with open( pathnameOriginal, 'r' ) as file:
        fileRead = file.readlines() 
    with open( 'pbrt/snow'+str(s)+'.pbrt', 'w') as f:
        for i, line in enumerate(fileRead):
            for j, word in enumerate(wordsToReplace):
                if word in line:
                    line = line.replace(word, replacement[j]) 
            f.write(line)
                
replacePbrt( pathnameOriginal, 0 )

In [7]:
pathname = r"/Users/daniellam/Desktop/materialPointOptimizedCopy/case1"
pathnameOriginal = r'/Users/daniellam/Downloads/pbrscenes/pbrt-v3-scenes/snow/geometry/density_render.70.pbrt'
pathnameOriginal2 = r"/Users/daniellam/Desktop/materialPointOptimizedCopy/snow41.pbrt"


filenames = []
    
for subdir, dirs, files in os.walk(pathname):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath[-3:] == 'npy':
                filenames.append(filepath)


sort_nicely(filenames)
N = len(filenames)//3

xpaths = []
upaths = []
mpaths = []


for i, path in enumerate(filenames):
    if i//N == 0:
        mpaths.append(path)
    if i//N == 1:
        upaths.append(path)
    if i//N == 2:
        xpaths.append(path)


g = 9.81
Nx = 50
Ny = 50
Nz = 50
eg = eulerianGrid(Nx, Ny, Nz)


i = 0
for xpath, upath, mpath in zip(xpaths, upaths, mpaths):
    try:
        lp
    except NameError:
        ## initialize lp
        X = np.load(xpath)
        numberParticles = X.shape[0]
        lp = lagrangianParticles(numberParticles)
        
        lp.loadData(xpath, upath, mpath)
        step1(eg, lp)
        writeData(pathnameOriginal, eg.mm, i)
        replacePbrt( pathnameOriginal2, i )
    else:        
        lp.loadData(xpath, upath, mpath)
        step1(eg, lp)
        writeData(pathnameOriginal, eg.mm, i)
        replacePbrt( pathnameOriginal2, i )
        
    print('pbrt ../../pbrt-v3-scenes/cloud/snow' + str(i) + '.pbrt' )
    i += 1








pbrt ../../pbrt-v3-scenes/cloud/snow0.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow1.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow2.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow3.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow4.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow5.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow6.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow7.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow8.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow9.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow10.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow11.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow12.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow13.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow14.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow15.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow16.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow17.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow18.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow19.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow20.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow21.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow22.pbr

pbrt ../../pbrt-v3-scenes/cloud/snow185.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow186.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow187.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow188.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow189.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow190.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow191.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow192.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow193.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow194.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow195.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow196.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow197.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow198.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow199.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow200.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow201.pbrt
pbrt ../../pbrt-v3-scenes/cloud/snow202.pbrt
